In [2]:
import pandas as pd
import numpy as np
import os
from matplotlib import pyplot as plt

In [3]:
FILES_PATH = "/home/guillaume/pythia/code/data/"

In [4]:
fixtures_data = pd.read_csv(f"{FILES_PATH}fixtures.csv", delimiter=",")
odds_data = pd.read_csv(f"{FILES_PATH}odds.csv", delimiter=",")
players_data = pd.read_csv(f"{FILES_PATH}players.csv", delimiter=",")
results_data = pd.read_csv(f"{FILES_PATH}results.csv", delimiter=",")
startingXI_data = pd.read_csv(f"{FILES_PATH}startingXI.csv", delimiter=",")
teams_data = pd.read_csv(f"{FILES_PATH}teams.csv", delimiter=",")

In [5]:
results_data = results_data.merge(teams_data, left_on="HomeTeamID", right_on = "TeamID", how="left")\
    .rename(columns = {"TeamName" : "HomeTeamName"})\
        .drop(columns="TeamID")

results_data = results_data.merge(teams_data, left_on="AwayTeamID", right_on = "TeamID", how="left")\
    .rename(columns = {"TeamName" : "AwayTeamName"})\
        .drop(columns="TeamID")

## Feature engineering on players

Let's first check if any players with the same name.

In [10]:
players_data[players_data["PlayerName"].duplicated()]
## It is important to make the distinctions between these players who share their name with at least another player.

,PlayerName,TeamID,PositionID
92,Frank Smith,12,3
414,Fred Williams,24,3
650,Robert Pitts,8,1


We create a unique ID for every player

In [11]:
players_data["PlayerID"] = np.arange(1, len(players_data)+1)

### Add team ID to the dataset using players' name

In [12]:
homonymous_player_names = players_data[players_data["PlayerName"].duplicated()]["PlayerName"].values
player_names = startingXI_data["StartingXI"][0].split(",")

def link_players_to_teamID(player_names : list, homonymous_player_names : list = homonymous_player_names)->str:
    for _name in player_names:
        if _name not in homonymous_player_names:
            return players_data[players_data["PlayerName"]==_name]["TeamID"].values[0]
        else: 
            continue

startingXI_data["StartingXI"] = startingXI_data["StartingXI"].map(lambda x: x.split(","))
startingXI_data["TeamID"] = startingXI_data["StartingXI"].map(lambda x: link_players_to_teamID(x))

### Explode the dataframe to get one player per row, per game, per team

In [13]:
startingXI_data_exploded = startingXI_data.copy()
startingXI_data_exploded = startingXI_data_exploded.explode("StartingXI").rename(columns = {"StartingXI" : "PlayerName"})
startingXI_data_exploded = startingXI_data_exploded.merge(players_data, on = ["PlayerName", "TeamID"], how = "left")
startingXI_data_exploded["MatchPlayed"] = 1 ## This variable will be useful later on for any calculations (e.g.: to count the number of games played)

### Check the number of formations used throughout the two seasons

In [14]:
formation_data = startingXI_data_exploded.groupby(["MatchID", "TeamID", "PositionID"]).agg(
    {"MatchPlayed" : "sum"}
).reset_index().sort_values(by=["MatchID", "TeamID", "PositionID"]).groupby(["MatchID", "TeamID"]).agg(
    {"MatchPlayed":list}
).reset_index().rename(columns={"MatchPlayed" : "MatchFormation"})

set(tuple(i) for i in formation_data["MatchFormation"])

{(1, 3, 5, 2), (1, 4, 3, 3), (1, 4, 4, 2)}

We add the number of defenders, midfielders, forwards for each team and every game

In [15]:
formation_data["NumberDefenders"] = formation_data.apply(lambda x: x.MatchFormation[1], axis=1)
formation_data["NumberMidfielders"] = formation_data.apply(lambda x: x.MatchFormation[2], axis=1)
formation_data["NumberForwards"] = formation_data.apply(lambda x: x.MatchFormation[3], axis=1)
formation_data

,MatchID,TeamID,MatchFormation,NumberDefenders,NumberMidfielders,NumberForwards
0,1,1,"[1, 4, 3, 3]",4,3,3
1,1,7,"[1, 4, 3, 3]",4,3,3
2,2,6,"[1, 3, 5, 2]",3,5,2
3,2,8,"[1, 4, 3, 3]",4,3,3
4,3,5,"[1, 4, 4, 2]",4,4,2
...,...,...,...,...,...,...
3019,1510,17,"[1, 3, 5, 2]",3,5,2
3020,1511,13,"[1, 4, 3, 3]",4,3,3
3021,1511,16,"[1, 4, 3, 3]",4,3,3
3022,1512,14,"[1, 4, 3, 3]",4,3,3


### Explore formation of players for every game

In [16]:
all_combinations = startingXI_data_exploded.sort_values(by=["MatchID", "TeamID", "PositionID", "PlayerID"]).groupby(["MatchID", "TeamID", "PositionID"]).agg(
    {"PlayerID" : list}
).reset_index().groupby(["MatchID", "TeamID"]).agg(
    {"PlayerID":list}
).reset_index()

all_combinations

,MatchID,TeamID,PlayerID
0,1,1,"[[2], [4, 6, 8, 9], [15, 17, 18], [20, 22, 25]]"
1,1,7,"[[628], [631, 634, 635, 636], [641, 642, 644],..."
2,2,6,"[[601], [604, 609, 611], [612, 613, 614, 617, ..."
3,2,8,"[[652], [654, 656, 660, 661], [664, 665, 668],..."
4,3,5,"[[578], [582, 584, 585, 586], [587, 588, 591, ..."
...,...,...,...
3019,1510,17,"[[203], [205, 206, 207], [212, 214, 215, 217, ..."
3020,1511,13,"[[101], [105, 108, 110, 111], [115, 116, 117],..."
3021,1511,16,"[[176], [179, 180, 184, 186], [189, 192, 193],..."
3022,1512,14,"[[128], [129, 131, 133, 136], [137, 138, 140],..."


In [17]:
all_combinations["StartingGoalkeeper"] = all_combinations.apply(lambda x: x.PlayerID[0], axis=1)
all_combinations["StartingDefenders"] = all_combinations.apply(lambda x: x.PlayerID[1], axis=1)
all_combinations["StartingMidfielders"] = all_combinations.apply(lambda x: x.PlayerID[2], axis=1)
all_combinations["StartingForwards"] = all_combinations.apply(lambda x: x.PlayerID[3], axis=1)
all_combinations = all_combinations.drop(columns = ["PlayerID"])

In [18]:
all_combinations = all_combinations.merge(results_data, on = ["MatchID"], how = "left")

We add the Boolean: was the game home or away?

In [19]:
all_combinations["Home"] = all_combinations.apply(lambda x : 1 if x.HomeTeamID ==x.TeamID else 0, axis = 1)

## Goalkeeper score calculation

In [20]:
team_data = all_combinations.copy()
team_data = team_data[team_data["TeamID"]==1]

In [21]:
total_list = []
for i in range(len(team_data)//4):
    total_list += 4*[i//4]

team_data["GameSeries"] = total_list

We merge the season results with the players dataset

In [236]:
results_with_players = startingXI_data_exploded.merge(results_data, on = ["MatchID"], how = "left")

We perform feature engineering on season 1 only

In [237]:
results_with_players = results_with_players[results_with_players["SeasonID"]==1]

Create new general variables for score and shots to avoid the home/away distinction

In [238]:
results_with_players["TeamScore"] = results_with_players.apply(lambda x : x.HomeScore if x.TeamID == x.HomeTeamID else x.AwayScore, axis=1)
results_with_players["OpponentScore"] = results_with_players.apply(lambda x : x.AwayScore if x.TeamID == x.HomeTeamID else x.HomeScore, axis=1)
results_with_players = results_with_players.drop(columns = ["HomeScore", "AwayScore"])


results_with_players["TeamShots"] = results_with_players.apply(lambda x : x.HomeShots if x.TeamID == x.HomeTeamID else x.AwayShots, axis=1)
results_with_players["OpponentShots"] = results_with_players.apply(lambda x : x.AwayShots if x.TeamID == x.HomeTeamID else x.HomeShots, axis=1)
results_with_players = results_with_players.drop(columns = ["HomeShots", "AwayShots"])


We calculate the number of goals and shots conceded for every player over the past 4 games they played

In [294]:
results_with_players = results_with_players.sort_values(by = ["PlayerID", "Gameweek"], ascending=True).reset_index(drop=True)

results_with_players["GoalkeeperConcededGoalsXGames"]  = results_with_players\
    .groupby("PlayerID")["OpponentScore"]\
        .apply(lambda x: x.shift(1)\
               .rolling(window=4, min_periods=1).sum())\
                .reset_index(level=0, drop=True)

results_with_players["GoalkeeperConcededShotsXGames"]  = results_with_players.groupby("PlayerID")["OpponentShots"]\
    .apply(lambda x: x.shift(1)\
           .rolling(window=4, min_periods=1).sum())\
            .reset_index(level=0, drop=True)

results_with_players["GoalkeeperConcededGoalsAllSeason"]  = results_with_players.groupby("PlayerID")["OpponentScore"]\
    .apply(lambda x: x.shift(1)\
           .cumsum())\
            .reset_index(level=0, drop=True)

results_with_players["GoalkeeperConcededShotsAllSeason"]  = results_with_players.groupby("PlayerID")["OpponentShots"]\
    .apply(lambda x: x.shift(1)\
           .cumsum())\
            .reset_index(level=0, drop=True)

# results_with_players["PlayerMatchPlayedXGames"] = results_with_players.groupby("PlayerID")["MatchPlayed"]\
#     .apply(lambda x: x.shift(1)\
#            .rolling(window=4, min_periods=1).sum())\
#             .reset_index(level=0, drop=True)

# results_with_players["PlayerMatchPlayedXGames"] = results_with_players.groupby("PlayerID")["MatchPlayed"].rolling(window=4, min_periods = 1).sum().reset_index(level=0, drop=True)
results_with_players["PlayerMatchPlayedXGames"] = results_with_players.groupby("PlayerID")["MatchPlayed"].cumsum().reset_index(level=0, drop=True)


In [295]:
results_with_players["GoalkeeperFirstGame"] = results_with_players["GoalkeeperConcededShotsAllSeason"].map(lambda x : 1 if pd.isna(x) else 0)

In [296]:
results_with_players["GoalkeeperConcededShotsXGames"] = results_with_players["GoalkeeperConcededShotsXGames"].fillna(0)
results_with_players["GoalkeeperConcededGoalsXGames"] = results_with_players["GoalkeeperConcededGoalsXGames"].fillna(0)

In [297]:
results_with_players["GoalkeeperRatioXGames"] = results_with_players.apply(lambda x : 1-x.GoalkeeperConcededGoalsXGames/x.GoalkeeperConcededShotsXGames if x.GoalkeeperConcededShotsXGames!=0 else 0, axis=1)

Compute weighted mean of goalkeeper rating

In [298]:
results_data_season1 = results_data.copy()
results_data_season1 = results_data_season1[results_data_season1["SeasonID"]==1]
results_data_season1["TotalShots"] = (results_data_season1["HomeShots"]+results_data_season1["AwayShots"])//2
results_data_season1["TotalDefenseRate"] = (results_data_season1["HomeDefenseRate"]+results_data_season1["AwayDefenseRate"])/2

In [299]:
results_data_season1["BaselineTotalShots"] = results_data_season1.groupby(["SeasonID", "Gameweek"])["TotalShots"].cumsum()//results_data_season1["Gameweek"]
results_data_season1["BaselineGoalkeeperRating"] = results_data_season1.groupby(["SeasonID", "Gameweek"])["TotalDefenseRate"].mean().reset_index(drop=True)

# /results_data_season1["Gameweek"]

In [ ]:
results_data_season1 = results_data_season1.groupby(["SeasonID", "Gameweek"]).agg(
  AverageDefenseRate = ("TotalDefenseRate", "mean"),
  AverageTotalShots = ("TotalShots", "mean")).reset_index(drop=False)

results_with_players = results_with_players.merge(results_data_season1, on = ["SeasonID", "Gameweek"], how="left")

## Plan of action
- $$\text{Goalkeeper rating} = (1 - \frac{\text{Goals conceded}}{\text{Number of shots conceded}})*R$$
- $$\text{R} = \frac{\text{Number of shots conceded}}{\text{Average number of shots conceded}}
- The variable Avergae number of shots conceded is a sliding window over the past X gameweeks
- That way we take into account the fact that a goalkeeper facing many shots and stopping most of them is probably better.

- $$\text{Adjusted rating} = \frac{n}{n+k}*\text{Goalkeeper rating} + \frac{k}{k+n}*\text{baseline rating}$$
- n: number of shots faced by the goalkeeper
- k: influence of the baseline
- baseline rating is the average rating of goalkeepers over a sliding window of size X


- Average of all the Goalkeeper ratings over time using the following weighted sum:
- Let's say it is gameweek 10, the goalkeeper played at Gameweeks: 3, 5, 7
- Weights = [3, 5, 7]
- $$\text{Normalized weights} = \frac{\text{weight}_{i}}{\text{Sum available weights}}$$
- That way we give more weight to the most recent performances of the goalkeeper.


- $$\text{Adjusted rating} = \frac{n}{n+k}*\text{Goalkeeper rating} + \frac{k}{k+n}*\text{baseline rating}$$
- n: number of games played by the player
- k: influence of the baseline
- baseline rating is the average rating of goalkeepers over a sliding window of size X
- That way we avoid giving too much influence to ratings based on a little number of games

In [309]:
results_with_players

,MatchID,PlayerName,TeamID,PositionID,PlayerID,MatchPlayed,SeasonID,Gameweek,HomeTeamID,AwayTeamID,...,OpponentShots,GoalkeeperConcededGoalsXGames,GoalkeeperConcededShotsXGames,GoalkeeperConcededGoalsAllSeason,GoalkeeperConcededShotsAllSeason,GoalkeeperFirstGame,GoalkeeperRatioXGames,PlayerMatchPlayedXGames,AverageDefenseRate,AverageTotalShots
0,15,Odell Williams,1,1,1,1,1,2,2,1,...,9,0.0,0.0,NaN,NaN,1,0.000000,1,0.912683,13.785714
1,43,Odell Williams,1,1,1,1,1,4,1,13,...,12,0.0,9.0,0.0,9.0,0,1.000000,2,0.883735,12.857143
2,183,Odell Williams,1,1,1,1,1,14,14,1,...,11,1.0,21.0,1.0,21.0,0,0.952381,3,0.864987,13.071429
3,197,Odell Williams,1,1,1,1,1,15,1,4,...,13,2.0,32.0,2.0,32.0,0,0.937500,4,0.905516,13.928571
4,267,Odell Williams,1,1,1,1,1,20,1,24,...,14,3.0,45.0,3.0,45.0,0,0.933333,5,0.883708,14.357143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16627,602,Isidro Swanson,9,4,700,1,1,43,10,9,...,9,4.0,51.0,29.0,257.0,0,0.921569,20,0.878747,11.142857
16628,614,Isidro Swanson,9,4,700,1,1,44,4,9,...,21,4.0,46.0,30.0,266.0,0,0.913043,21,0.894322,13.071429
16629,696,Isidro Swanson,9,4,700,1,1,50,18,9,...,9,6.0,56.0,33.0,287.0,0,0.892857,22,0.899174,13.071429
16630,718,Isidro Swanson,9,4,700,1,1,52,15,9,...,32,5.0,54.0,33.0,296.0,0,0.907407,23,0.870724,13.285714


In [ ]:
results_with_players[(results_with_players["TeamID"]==1) & (results_with_players["PlayerID"]==2)][["OpponentScore", "GoalkeeperConcededGoalsAllSeason"]]